In [12]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

system_message = """
    你是一个标签提取专家。请从项目的文本信息中提取结构化信息，输入的项目并按要求输出标签。
    ---
    【支持的标签类型】
    - 领域
    - 行业
    ---
    【标签解释】
    - 领域：项目所属的细分领域，先在给定的领域集中筛选，如果项目所属领域不属于给定的领域集，则请忽略给定的领域集，自行通过推理为该项目生成一个领域名称。
        如果为项目自行生成领域名称，请根据项目情况完成推理，生成一个合适的名称，要求所生成的领域名称合理、有效。
        项目可能的领域集包含如下领域：工业产业园、基础设施提档升级、农业产业园、棚户区改造、医院建设、林草、城市电网、铁路、生活垃圾、改扩提标、水厂工程、轨道交通、政设施提档升级、机场、安居工程、基础设施改造提升、充电桩、物流基础设施提升、闸坝工程、水环境综合治理、码头工程、能源、公共交通提档升级、交通设施体系提升、公交更新、殡葬服务、智慧港口、管线整治、提标扩能、电商产业园、公共卫生、医院信息化、市政、智慧公路、保障住房、油茶加工、产业链、市场监控、国土综合整治、现代渔业、灌区续建配套与节水改造、区县数公基建设、养老服务、 中央储备粮粮库升级改造、数据要素平台建设、数智化病理服务、医共体、传染病监测预警、医院服务能力提升、执法办案中心、供水管网改造、饮水工程、工业智能改造、校园安全监控接入上收、城市生命线、排水防涝、燃气管网改造、智慧停车、冷链物流、供应链、公共交通提档升级、高标准农田、人居环境、田园综合体、乡村振兴、生态金融、景区提升、路灯节能更新改造、重点项目、数字档案馆、成品油监管、大田并小田、智慧水务、智慧水厂、老旧小区改造、污水处理、流域治理
    - 行业：项目所属的行业主体，先在给定的行业范围中进行匹配，如果项目所属行业不在给定的行业范围中，则请忽略给定范围，自行通过推理为该项目生成一个行业名称。
        如果为项目自行生成行业名称，请根据项目情况完成推理，生成一个合适的名称，要求所生成的行业名称合理、有效。
        项目可能的行业范围包含如下行业：政务、住建、工业、战客、政法、公安、交通、教育、卫健、商客、金融、农业
        
    ---
    【输出要求】
    1. 请用 JSON 格式输出，如：[{"key": "领域", "value": "冷链物流"}, {"key": "行业", "value": "工业"}]
    2. 标签为纯文本内容，请不要加入特殊字符（如转义字符）
    3. 不要生成“其他”和“无法确定”等标签
    4. 不要输出换行符"\n"和空格
    ---
    待分析文本如下：
"""

def extract_tags(text):
    """
    对文本数据提取标签

    输出格式为 JSON 格式
    """
    completion = client.chat.completions.create(
        model = "qwen-max",
        messages = [
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': text}
        ],
        response_format = {"type": "json_object"}
    )
    return completion.choices[0].message.content

In [13]:
# 测试：项目分类问题
program_text = """
| 农林水利 | 湖北麻城数字农业产业园项目 | 本项目建设地点位于位于中馆驿镇，建设内容：<br>规划总用地面积约517 亩，总建筑面积233857平方米，其中：种植智能化玻璃温室4栋，建筑面积175500平方米；种植拱棚10栋，建筑面积34282平方米；育苗玻璃温室1栋，建筑面积12150平方米；种植配套2栋，建筑面积10305平方米；育苗配套1栋，建筑面积1620平方米。并配套建设道路、排水、给水、电力、停车场、传达室、公厕等。园区整体方面以数字化高科技种植型智能化玻璃温室主体，兼顾休闲观光采摘、园艺、展示为主题的综合型数字农业园区。 | 湖北举润农产品开发有限公司 |
"""
print(program_text)
print("\n项目分类问题标签提取结果：")
label = extract_tags(program_text)
print(label)


| 农林水利 | 湖北麻城数字农业产业园项目 | 本项目建设地点位于位于中馆驿镇，建设内容：<br>规划总用地面积约517 亩，总建筑面积233857平方米，其中：种植智能化玻璃温室4栋，建筑面积175500平方米；种植拱棚10栋，建筑面积34282平方米；育苗玻璃温室1栋，建筑面积12150平方米；种植配套2栋，建筑面积10305平方米；育苗配套1栋，建筑面积1620平方米。并配套建设道路、排水、给水、电力、停车场、传达室、公厕等。园区整体方面以数字化高科技种植型智能化玻璃温室主体，兼顾休闲观光采摘、园艺、展示为主题的综合型数字农业园区。 | 湖北举润农产品开发有限公司 |


项目分类问题标签提取结果：
[{"key":"领域","value":"农业产业园"},{"key":"行业","value":"农业"}]


In [6]:
# 数据准备
import pandas as pd 
 
# 读取Excel文件（需根据实际路径修改）
file_path = './Data/tmpfile.xlsx'   # 替换为实际文件路径 
df = pd.read_excel( 
    file_path,
    names=['项目类型', '项目名称', '项目简述', '承建商'],  # 指定列名 
    engine='openpyxl'  # 处理xlsx格式 
)

In [7]:
data = ["test", "test2"]
for _, row in df.iterrows(): 
    project_str = (
        f"|项目类型：{row['项目类型'] or '暂无信息'}"
        f"|项目名称：{row['项目名称'] or '暂无信息'}"
        f"|项目简述：{row['项目简述'] or '暂无信息'}。"
        f"|承建商：{row['承建商'] or '暂无信息'}|"
    )
    data.append(project_str)

data = data[2:]
# 验证数据（可选）
print(f"生成{len(data)}条项目信息：")
for item in data[:6]:  # 展示前6条样例 
    print(item)

生成1309条项目信息：
|项目类型：生态环保|项目名称：葛店全域生态涵养建设项目|项目简述：建设地点：鄂州市葛店经济技术开发区
（1）交通优化、建筑整治改造、环境设施提升、基础设施升级：沿葛洪大道、建设大道、高新大道、创业大道道路交通优化、建筑整治改造、环境设施提升、基础设施升级、增设广告牌等。
（2）配套基础设施及停车场、人行步道建设：对长江岸线、武九铁路（葛店段）两侧、武鄂高速（葛店段）两侧、高新大道两侧、紫菱湖路及沿湖路两侧建设配套基础设施及停车场、人行步道、增设广告牌等。
（3）道路、人行道改造：对兆麟大道、人民西路、高新四路、高新五路、高新二路、新华大道、发展大道、建设大道、创业大道、东湖大道、陶塘大道进行道路、人行道改造、增设广告牌。。|承建商：湖北省葛店开发区科技新城建设投资有限公司|
|项目类型：市政和产业园区基础设施|项目名称：监利市城乡综合物流及智慧交通体系建设项目|项目简述：实施方：监利市丰盈交通投资有限公司
建设地点：监利市容城镇、朱河镇、网市镇、汪桥镇、柘木乡、黄歇口镇、尺八镇总占地面积约361.7亩，新建容城镇港口物流园212398㎡、改造朱河镇等6个乡镇综合运输服务站10613.45㎡；智慧交通体系创建5个应用端、4个应用平台、1个大数据仓、3套支撑平台、2套保障体系；改造提升10条城乡公路全长106.74km、宽6.5-8.5m；配套物流设施等。。|承建商：中交特种工程有限公司|
|项目类型：市政和产业园区基础设施|项目名称：湖北点军工业园区北斗产业基础设施建设项目|项目简述：项目实施方：宜昌金辉大数据产业发展有限公司
建设地点：宜昌市点军区桥边镇李家湾村
北斗产业孵化楼、研发制造楼、公寓、食堂、架空层（车库）、人防地下车库，配套水、电、道路、给排水、景观绿化、大门及围墙、广场、空中花园等；
同时设置地面停车位及车库停车位。建设园内配套市政道路7条。。|承建商：中标候选人：
湖北鑫民建设集团有限公司、湖北省清江路桥建筑有限公司、湖北中广公路勘察设计有限公司联合体
湖北新阳市政工程有限公司、武汉市政工程设计研究院有限责任公司联合体
固川建设集团有限公司、济南市市政工程设计研究院（集团）有限责任公司联合体|
|项目类型：市政和产业园区基础设施|项目名称：枣阳国家级经济技术开发区核心区基础设施提档升级建设项目|项目简述：项目建设规模:新建

In [8]:
# 开始项目分类问题
import time

labels = []
for item in data: 
    label = extract_tags(item)
    labels.append(label)

In [9]:
print(len(labels))
print(labels[0])
print(labels[1])
print(labels[2])

1309
[{"key":"领域","value":"基础设施提档升级"},{"key":"行业","value":"住建"}]
[{"key":"领域","value":"物流基础设施提升"},{"key":"行业","value":"交通"}]
[{"key":"领域","value":"工业产业园"},{"key":"行业","value":"工业"}]


In [10]:
import json
with open('Qwen_72B.json',  'w', encoding = 'utf-8') as f:
    json.dump(labels,  f, ensure_ascii=False)

In [ ]:
# with open('output.txt',  'w') as f:
#     f.write(str(labels))   # 需将变量转为字符串[2]()

In [ ]:
prompt_optimization = """
    您是一位标签提取专家，任务是从给定项目的文本信息中提取结构化信息，并按照要求输出相应的标签。请根据以下指导完成任务。

    ### 任务背景
    - 您需要从项目描述中识别出两个关键标签：领域和行业。
    - 这些标签对于理解项目的性质及其所属的经济活动范围至关重要。

    ### 支持的标签类型
    - **领域**：指项目所属的具体细分领域。
    - **行业**：指项目所处的主要行业类别。

    ### 标签定义与处理规则
    - **领域**：
    - 首先尝试在预定义的领域列表中找到匹配项。如果找不到，则基于项目内容自行推理出一个合理的领域名称。
    - 给定的领域集包括但不限于：工业产业园、基础设施提档升级、农业产业园、棚户区改造、医院建设、林草、城市电网、铁路、生活垃圾、改扩提标、水厂工程、轨道交通、政设施提档升级、机场、安居工程、基础设施改造提升、充电桩、物流基础设施提升、闸坝工程、水环境综合治理、码头工程、能源、公共交通提档升级、交通设施体系提升、公交更新、殡葬服务、智慧港口、管线整治、提标扩能、电商产业园、公共卫生、医院信息化、市政、智慧公路、保障住房、油茶加工、产业链、市场监控、国土综合整治、现代渔业、灌区续建配套与节水改造、区县数公基建设、养老服务、 中央储备粮粮库升级改造、数据要素平台建设、数智化病理服务、医共体、传染病监测预警、医院服务能力提升、执法办案中心、供水管网改造、饮水工程、工业智能改造、校园安全监控接入上收、城市生命线、排水防涝、燃气管网改造、智慧停车、冷链物流、供应链、公共交通提档升级、高标准农田、人居环境、田园综合体、乡村振兴、生态金融、景区提升、路灯节能更新改造、重点项目、数字档案馆、成品油监管、大田并小田、智慧水务、智慧水厂、老旧小区改造、污水处理、流域治理
    - **行业**：
    - 同样地，首先尝试在预定义的行业范围内寻找匹配项。如果没有合适的匹配，则依据项目具体情况推断出合适的行业名称。
    - 可能涉及的行业范围有：政务、住建、工业、战客、政法、公安、交通、教育、卫健、商客、金融、农业

    ### 输入
    - 项目文本信息：`${project_text}`

    ### 输出格式
    - 请以JSON格式返回结果，示例如下：
    ```
    [
        {"key": "领域", "value": "领域名称"},
        {"key": "行业", "value": "行业名称"}
    ]
    ```

    ### 注意事项
    - 确保生成的领域和行业名称合理且准确反映项目特性。
    - 如果无法直接从给定列表中找到匹配项，请基于项目内容进行合理推测。
    - 输出必须严格按照指定格式组织。

    请严格按照上述指示执行任务。
"""